# Fitting a Carbonate Correction for MP-sourced Gibbs free energy of formation data (300 K)

Author: Matthew McDermott

Date: May 24, 2022

#### Imports

In [4]:
from emmet.core.thermo import ThermoType
from mp_api.client import MPRester
from pymatgen.core.composition import Composition, Element
from pymatgen.entries.mixing_scheme import MaterialsProjectDFTMixingScheme

import numpy as np
import pandas
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import minimize

from rxn_network.entries.entry_set import GibbsEntrySet

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Chemical systems

In [5]:
metals = ["Ag", "Ba", "Ca", "Cd", "Cs", "Fe", "K", 
          "Li", "Mg", "Mn", "Na", "Pb", "Rb", "Sr", "Zn"]  # all metals with carbonate energies available

systems = [(m, "C", "O") for m in metals]

### Downloading Materials Project (MP) data using API

In [6]:
all_entries = {}

for system in systems:
    with MPRester() as mpr:
        entries = mpr.get_entries_in_chemsys(
            elements=system,
            compatible_only=False,
            additional_criteria={
                "thermo_types": [ThermoType.GGA_GGA_U, ThermoType.R2SCAN]
            },
        )
        # TODO: waiting for mp-api fix, then can remove this in-place mixing
    entries = MaterialsProjectDFTMixingScheme().process_entries(entries)
        
    all_entries[system[0]] = entries

Retrieving ThermoDoc documents:   0%|          | 0/182 [00:00<?, ?it/s]

Processing 140 GGA(+U) and 42 R2SCAN entries...
  Processed 140 compatible GGA(+U) entries with MaterialsProject2020Compatibility
  Entries belong to the {'O', 'Ag', 'C'} chemical system
  Generating mixing state data from provided entries.


Failed to guess oxidation states for Entry mp-1214896-GGA (AgO2). Assigning anion correction to only the most electronegative atom.
Failed to guess oxidation states for Entry mp-27819-GGA (Ag3O). Assigning anion correction to only the most electronegative atom.
Failed to guess oxidation states for Entry mp-1197923-GGA (C61O2). Assigning anion correction to only the most electronegative atom.


  Entries contain R2SCAN calculations for 7 of 8 GGA(+U) hull entries.
  R2SCAN energies will be adjusted to the GGA(+U) scale
  The energy above hull for R2SCAN materials at compositions with R2SCAN hull entries will be preserved. For other compositions, Energies of R2SCAN materials will be set equal to those of matching GGA(+U) materials
  Discarding GGA entry mp-47-GGA for C4 because there is a matching R2SCAN material
  Discarding GGA entry mp-644607-GGA for C2 O4 because there is a matching R2SCAN material
  Discarding GGA entry mp-124-GGA for Ag1 because there is a matching R2SCAN material
  Discarding GGA entry mp-8566-GGA for Ag4 because there is a matching R2SCAN material
  Discarding GGA entry mp-990448-GGA for C2 because there is a matching R2SCAN material
  Discarding GGA entry mp-611448-GGA for C12 because there is a matching R2SCAN material
  Discarding GGA entry mp-606949-GGA for C12 because there is a matching R2SCAN material
  Discarding GGA entry mp-1190699-GGA for C8

WARNING! Discarding GGA entry mp-1065697-GGA for O1 because it was not found in the mixing state data. This can occur when there are duplicate structures. In such cases, only the lowest energy entry with that structure appears in the mixing state data.
WARNING! Discarding GGA entry mp-611836-GGA for O2 because it was not found in the mixing state data. This can occur when there are duplicate structures. In such cases, only the lowest energy entry with that structure appears in the mixing state data.
WARNING! Discarding GGA entry mp-1023923-GGA for O6 because it was not found in the mixing state data. This can occur when there are duplicate structures. In such cases, only the lowest energy entry with that structure appears in the mixing state data.
WARNING! Discarding GGA entry mp-1180050-GGA for O8 because it was not found in the mixing state data. This can occur when there are duplicate structures. In such cases, only the lowest energy entry with that structure appears in the mixing s

...

Processing 138 GGA(+U) and 34 R2SCAN entries...
  Processed 138 compatible GGA(+U) entries with MaterialsProject2020Compatibility
  Entries belong to the {'O', 'Zn', 'C'} chemical system
  Generating mixing state data from provided entries.


Failed to guess oxidation states for Entry mp-971857-GGA (Zn3C). Assigning anion correction to only the most electronegative atom.
Failed to guess oxidation states for Entry mp-981336-GGA (Zn3C). Assigning anion correction to only the most electronegative atom.
Failed to guess oxidation states for Entry mp-1197923-GGA (C61O2). Assigning anion correction to only the most electronegative atom.


  Entries contain R2SCAN calculations for 5 of 6 GGA(+U) hull entries.
  R2SCAN energies will be adjusted to the GGA(+U) scale
  The energy above hull for R2SCAN materials at compositions with R2SCAN hull entries will be preserved. For other compositions, Energies of R2SCAN materials will be set equal to those of matching GGA(+U) materials
  Discarding GGA entry mp-47-GGA for C4 because there is a matching R2SCAN material
  Discarding GGA entry mp-2133-GGA for Zn2 O2 because it is a GGA(+U) ground state that matches a R2SCAN material.
  Discarding GGA entry mp-644607-GGA for C2 O4 because there is a matching R2SCAN material
  Discarding GGA entry mp-990448-GGA for C2 because there is a matching R2SCAN material
  Discarding GGA entry mp-1986-GGA for Zn1 O1 because there is a matching R2SCAN material
  Discarding GGA entry mp-611448-GGA for C12 because there is a matching R2SCAN material
  Discarding GGA entry mp-606949-GGA for C12 because there is a matching R2SCAN material
  Discarding

WARNING! Discarding GGA entry mp-1065697-GGA for O1 because it was not found in the mixing state data. This can occur when there are duplicate structures. In such cases, only the lowest energy entry with that structure appears in the mixing state data.
WARNING! Discarding GGA entry mp-611836-GGA for O2 because it was not found in the mixing state data. This can occur when there are duplicate structures. In such cases, only the lowest energy entry with that structure appears in the mixing state data.
WARNING! Discarding GGA entry mp-1023923-GGA for O6 because it was not found in the mixing state data. This can occur when there are duplicate structures. In such cases, only the lowest energy entry with that structure appears in the mixing state data.
WARNING! Discarding GGA entry mp-1180050-GGA for O8 because it was not found in the mixing state data. This can occur when there are duplicate structures. In such cases, only the lowest energy entry with that structure appears in the mixing s

  Discarding R2SCAN entry mp-937760-R2SCAN for C4 because there is no matching GGA(+U) entry and no R2SCAN ground state at this composition.

Processing complete. Mixed entries contain 102 GGA(+U) and 31 R2SCAN entries.

entry_id    formula     spacegroup  run_type  eV/atom corr/atom e_above_hull
mp-2516584-GGAC           R-3m        GGA       -9.229  0.000     0.000    
mp-569304-R2SCANC           R-3m        R2SCAN    -9.227  0.711     0.002    
mp-169-R2SCANC           C2/m        R2SCAN    -9.225  0.721     0.003    
mp-937760-GGAC           Fmmm        GGA       -9.223  0.000     0.006    
mp-568286-GGAC           Cmme        GGA       -9.220  0.000     0.008    
mp-568363-GGAC           Cmme        GGA       -9.220  0.000     0.008    
mp-48-R2SCANC           P6_3/mmc    R2SCAN    -9.220  0.724     0.008    
mp-990448-R2SCANC           P6/mmm      R2SCAN    -9.220  0.715     0.009    
mp-1040425-GGAC           P6/mmm      GGA       -9.219  0.000     0.009    
mp-997182-R2SCANC   

### Acquiring GibbsComputedEntries and corresponding experimental entries

In [24]:
mp_entries = []
exp_entries = []

temp = 300  # Kelvin

for m, entries in all_entries.items():
    gibbs = GibbsEntrySet.from_computed_entries(entries, temp, include_nist_data=False, apply_carbonate_correction=False)
    gibbs_exp = GibbsEntrySet.from_computed_entries(entries, temp, include_nist_data=True, include_freed_data=True, apply_carbonate_correction=False)
    
    oxidation_states = Element(m).common_oxidation_states
    
    for os in oxidation_states:
        formula = Composition(f"{m}2(CO3){os}").reduced_formula

        mp_entry = None
        exp_entry = None
        
        try:
            mp_entry = gibbs.get_min_entry_by_formula(formula)
            exp_entry = gibbs_exp.get_min_entry_by_formula(formula)
        except:
            continue
        
        if exp_entry.is_experimental:
            mp_entries.append(mp_entry)
            exp_entries.append(exp_entry)

In [25]:
energies_calc = []
energies_exp = []
names = []

for e_exp, e_calc in zip(exp_entries, mp_entries):
    formula = e_exp.composition.reduced_formula
    num_atoms_per_fu = Composition(formula).num_atoms
    
    energy_exp = e_exp.energy_per_atom * num_atoms_per_fu
    energy_calc = e_calc.energy_per_atom * num_atoms_per_fu
    name = e_calc.composition.reduced_formula
    
    energies_exp.append(energy_exp)
    energies_calc.append(energy_calc)
    names.append(name)

### Plot before correction

In [26]:
df = pandas.DataFrame({"x":energies_exp, "y":energies_calc, "name":names})

scatter = px.scatter(df, x="x", y="y", hover_name="name", symbol="name", color="name")
line = px.line(x=[-12, -4], y=[-12, -4])

fig = go.Figure(scatter.data + line.data)
fig.update_layout({"title": "Carbonate formation energies",
                   "xaxis_title":"Experimental dGf (eV/f.u.)",
                   "yaxis_title":"Calculated dGf (eV/f.u.)"})
fig.show()

### Fitting correction with scipy.optimize.minimize
Minimizing the mean absolute error (MAE) for all corrected phases.

In [27]:
def func(correction):
    return np.mean(np.abs((df["y"] + correction) - df["x"]))  # definition of MAE

In [28]:
final_correction = minimize(func, 0.25).x[0]

In [29]:
df_new = df.copy()
df_new["y"] = df_new["y"] + final_correction

scatter_new = px.scatter(df_new, x="x", y="y", hover_name="name", symbol="name", color="name")
line_new = px.line(x=[-12, -4], y=[-12, -4])

fig_new = go.Figure(scatter_new.data + line_new.data)
fig_new.update_layout({"title": "Corrected carbonate formation energies",
                   "xaxis_title":"Experimental dGf (eV/f.u.)",
                   "yaxis_title":"Calculated dGf (eV/f.u.)"})
fig_new.show()

## Final correction value (eV per CO$_3^{2-}$):

In [30]:
print(final_correction)

0.7878732894247907


### Test carbonate correction class

In [31]:
li_entries = GibbsEntrySet.from_entries(all_entries["Li"], 300, include_nist_data=False)

/Users/mcdermott/PycharmProjects/reaction-network/src/rxn_network/entries/entry_set.py:515: FutureWarning:

This method is deprecated. Use from_computed_entries instead.



In [32]:
li2co3 = li_entries.get_min_entry_by_formula("Li2CO3")
li2co3.energy_adjustments

[ConstantEnergyAdjustment:
   Name: Gibbs SISSO Correction
   Value: 1.919 eV
   Uncertainty: 0.600 eV
   Description: Gibbs correction: dGf(300 K) - dHf (298 K) (1.919 eV)
   Generated by: None]

In [33]:
mp_entries = []
exp_entries = []

temp = 300  # Kelvin

for m, entries in all_entries.items():
    gibbs = GibbsEntrySet.from_entries(entries, temp, include_nist_data=False, apply_carbonate_correction=True)
    gibbs_exp = GibbsEntrySet.from_entries(entries, temp, include_nist_data=True, include_freed_data=True, apply_carbonate_correction=False)
    
    oxidation_states = Element(m).common_oxidation_states
    
    for os in oxidation_states:
        formula = Composition(f"{m}2(CO3){os}").reduced_formula

        mp_entry = None
        exp_entry = None
        
        try:
            mp_entry = gibbs.get_min_entry_by_formula(formula)
            exp_entry = gibbs_exp.get_min_entry_by_formula(formula)
        except:
            continue
        
        if exp_entry.is_experimental:
            mp_entries.append(mp_entry)
            exp_entries.append(exp_entry)

In [34]:
energies_calc = []
energies_exp = []
names = []

for e_exp, e_calc in zip(exp_entries, mp_entries):
    energy_exp = e_exp.energy_per_atom
    energy_calc = e_calc.energy_per_atom
    name = e_calc.composition.reduced_formula
    
    energies_exp.append(energy_exp)
    energies_calc.append(energy_calc)
    names.append(name)
    
df = pandas.DataFrame({"x":energies_exp, "y":energies_calc, "name":names})

scatter = px.scatter(df, x="x", y="y", hover_name="name", symbol="name", color="name")
line = px.line(x=[-2.75, -0.75], y=[-2.75, -0.75])

fig = go.Figure(scatter.data + line.data)
fig.update_layout({"title": "Test of Carbonate Correction",
                   "xaxis_title":"Experimental dGf (eV/atom)",
                   "yaxis_title":"Calculated dGf (eV/atom)"})
fig.show()